In [ ]:
%matplotlib inline
import numpy as np
import cv2
import os
from skimage.color import label2rgb
import random
from matplotlib import pyplot as plt
from albumentations import (
    PadIfNeeded,
    HorizontalFlip,
    VerticalFlip,    
    CenterCrop,    
    Crop,
    Compose,
    Transpose,
    RandomRotate90,
    ElasticTransform,
    GridDistortion, 
    OpticalDistortion,
    RandomSizedCrop,
    OneOf,
    CLAHE,
    RandomBrightnessContrast,    
    RandomGamma    
)



def strong_aug(p=.5):
    return Compose([
        RandomRotate90(),
        Flip(),
        Transpose(),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        OneOf([
            MotionBlur(p=.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomBrightnessContrast(),            
        ], p=0.3),
        HueSaturationValue(p=0.3),
    ], p=p)




def augment_flips_color(p=.5):
    return Compose([
        CLAHE(),
        RandomRotate90(),
        Transpose(),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50, rotate_limit=45, p=.75),
        Blur(blur_limit=3),
        OpticalDistortion(),
        GridDistortion(),
        HueSaturationValue()
    ], p=p)



def augment_and_show(aug, image, mask=None, bboxes=[], categories=[], category_id_to_name=[], filename=None, 
                     font_scale_orig=0.35, 
                     font_scale_aug=0.35, show_title=True, **kwargs):
    
    augmented = aug(image=image, mask=mask, bboxes=bboxes, category_id=categories)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_aug = cv2.cvtColor(augmented['image'], cv2.COLOR_BGR2RGB)

    for bbox in bboxes:
        visualize_bbox(image, bbox, **kwargs)

    for bbox in augmented['bboxes']:
        visualize_bbox(image_aug, bbox, **kwargs)

    if show_title:
        for bbox,cat_id in zip(bboxes, categories):
            visualize_titles(image, bbox, category_id_to_name[cat_id], font_scale=font_scale_orig, **kwargs)
        for bbox,cat_id in zip(augmented['bboxes'], augmented['category_id']):
            visualize_titles(image_aug, bbox, category_id_to_name[cat_id], font_scale=font_scale_aug, **kwargs)

    
    if mask is None:
        f, ax = plt.subplots(1, 2, figsize=(16, 8))
        aa=0
        #ax[0].imshow(image)
        #x[0].set_title('Original image')
        
        #ax[1].imshow(image_aug)
        #ax[1].set_title('Augmented image')
    else:
        #f, ax = plt.subplots(2, 2, figsize=(16, 16))
        
        if len(mask.shape) != 3:
            mask = label2rgb(mask, bg_label=0)            
            mask_aug = label2rgb(augmented['mask'], bg_label=0)
        else:
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
            mask_aug = cv2.cvtColor(augmented['mask'], cv2.COLOR_BGR2RGB)
            
        #ax[0, 0].imshow(image)
        #ax[0, 0].set_title('Original image')
        
        #ax[0, 1].imshow(image_aug)
        #ax[0, 1].set_title('Augmented image')
        
        #ax[1, 0].imshow(mask ,interpolation='nearest')
        #ax[1, 0].set_title('Original mask')
        #ax[1, 1].imshow(mask_aug ,interpolation='nearest')
        #ax[1, 1].set_title('Augmented mask')

    #f.tight_layout()
    if filename is not None:
        f.savefig(filename)
    #cv2.imwrite('C:/Users/genju/Desktop/BIG_NN/AugFr/'+str(0)+'.png', image_aug)
    #dd = cv2.imread(mask_aug)
    #cv2.imwrite('C:/Users/genju/Desktop/BIG_NN/AugMasks/'+str(0)+'.png', dd)    
    return augmented['image'], augmented['mask'], augmented['bboxes']




def loadimages(path = "."):
    return [os.path.join(path, f) for f in sorted(os.listdir(path)) if f.endswith('.png')]

path1='FrPNG'
path2='Masks'
filenamess=loadimages(path1)
filemasks=loadimages(path2)
index = 0
i=8
j=5
w=2
k=1
num=0

for file in filenamess:
    #Load frame
    # imagee = cv2.imread('FrPNG/Fr001.png')
    # maske = cv2.imread('Masks/Mask0000.png', 0)
    imagee = cv2.imread(filenamess[index])
    maske = cv2.imread(filemasks[index], 0)
    index = index +1
    image = np.copy(imagee)
    mask = np.copy(maske)
    maskk = np.copy(maske)
    original_height, original_width = image.shape[:2]

    #padifneeded
    aug = PadIfNeeded(p=1, min_height=128, min_width=128)
    augmented = aug(image=image, mask=mask)
    image_padded = augmented['image']
    mask_padded = augmented['mask']


    #1
    aug = VerticalFlip(p=1)
    augmented = aug(image=image, mask=mask)
    image_v_flipped = augmented['image']
    mask_v_flipped = augmented['mask']
    name=str(num)+str(k)+str(w)+str(j)+str(i)
    cv2.imwrite('AugFr/'+str(name)+'.png', image_v_flipped)
    cv2.imwrite('AugMasks/'+str(name)+'.png', mask_v_flipped)
    i=i+1
    if i==10:
        i=0
        j=j+1
    if j==10:
        j=0
        w=w+1
    if w==10:
        w=0
        k=k+1
    if k==10:
        k=0
        num=num+1
        
        
        
    #2
    aug = HorizontalFlip(p=1)
    augmented = aug(image=image, mask=mask)
    image_h_flipped = augmented['image']
    mask_h_flipped = augmented['mask']
    name=str(num)+str(k)+str(w)+str(j)+str(i)
    cv2.imwrite('AugFr/'+str(name)+'.png', image_h_flipped)
    cv2.imwrite('AugMasks/'+str(name)+'.png', mask_h_flipped)
    i=i+1
    if i==10:
        i=0
        j=j+1
    if j==10:
        j=0
        w=w+1
    if w==10:
        w=0
        k=k+1
    if k==10:
        k=0
        num=num+1
        
        
        
    #3
    aug = RandomRotate90(p=1)
    augmented = aug(image=image, mask=mask)
    image_rot90 = augmented['image']
    mask_rot90 = augmented['mask']
    name=str(num)+str(k)+str(w)+str(j)+str(i)
    cv2.imwrite('AugFr/'+str(name)+'.png', image_rot90)
    cv2.imwrite('AugMasks/'+str(name)+'.png', mask_rot90)
    i=i+1
    if i==10:
        i=0
        j=j+1
    if j==10:
        j=0
        w=w+1
    if w==10:
        w=0
        k=k+1
    if k==10:
        k=0
        num=num+1
        
        
        
        
    #4
    aug = Transpose(p=1)
    augmented = aug(image=image, mask=mask)
    image_transposed = augmented['image']
    mask_transposed = augmented['mask']
    name=str(num)+str(k)+str(w)+str(j)+str(i)
    cv2.imwrite('AugFr/'+str(name)+'.png', image_transposed)
    cv2.imwrite('AugMasks/'+str(name)+'.png', mask_transposed)
    i=i+1
    if i==10:
        i=0
        j=j+1
    if j==10:
        j=0
        w=w+1
    if w==10:
        w=0
        k=k+1
    if k==10:
        k=0
        num=num+1

    #5
    aug = ElasticTransform(p=1, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03)
    augmented = aug(image=image, mask=mask)
    image_elastic = augmented['image']
    mask_elastic = augmented['mask']
    name=str(num)+str(k)+str(w)+str(j)+str(i)
    cv2.imwrite('AugFr/'+str(name)+'.png', image_elastic)
    cv2.imwrite('AugMasks/'+str(name)+'.png', mask_elastic)
    i=i+1
    if i==10:
        i=0
        j=j+1
    if j==10:
        j=0
        w=w+1
    if w==10:
        w=0
        k=k+1
    if k==10:
        k=0
        num=num+1
        
        
        
    #6
    aug = GridDistortion(p=1)
    augmented = aug(image=image, mask=mask)
    image_grid = augmented['image']
    mask_grid = augmented['mask']
    name=str(num)+str(k)+str(w)+str(j)+str(i)
    cv2.imwrite('AugFr/'+str(name)+'.png', image_grid)
    cv2.imwrite('AugMasks/'+str(name)+'.png', mask_grid)
    i=i+1
    if i==10:
        i=0
        j=j+1
    if j==10:
        j=0
        w=w+1
    if w==10:
        w=0
        k=k+1
    if k==10:
        k=0
        num=num+1

    # #7
    # aug = OpticalDistortion(p=1, distort_limit=2, shift_limit=0.5)
    # augmented = aug(image=image, mask=mask)
    # image_optical = augmented['image']
    # mask_optical = augmented['mask']
    # name=str(num)+str(k)+str(w)+str(j)+str(i)
    # cv2.imwrite('AugFr/'+str(name)+'.png', image_optical)
    # cv2.imwrite('AugMasks/'+str(name)+'.png', mask_optical)
    # i=i+1
    # if i==10:
    #     i=0
    #     j=j+1
    # if j==10:
    #     j=0
    #     w=w+1
    # if w==10:
    #     w=0
    #     k=k+1
    # if k==10:
    #     k=0
    #     num=num+1

    # #8
    # aug = RandomSizedCrop(p=1, min_max_height=(50, 101), height=original_height, width=original_width)
    # augmented = aug(image=image, mask=mask)
    # image_scaled = augmented['image']
    # mask_scaled = augmented['mask']
    # name=str(num)+str(k)+str(w)+str(j)+str(i)
    # cv2.imwrite('AugFr/'+str(name)+'.png', image_scaled)
    # cv2.imwrite('AugMasks/'+str(name)+'.png', mask_scaled)
    # i=i+1
    # if i==10:
    #     i=0
    #     j=j+1
    # if j==10:
    #     j=0
    #     w=w+1
    # if w==10:
    #     w=0
    #     k=k+1
    # if k==10:
    #     k=0
    #     num=num+1

    #9
    aug = Compose([VerticalFlip(p=0.5),              
                  RandomRotate90(p=0.5)])
    augmented = aug(image=image, mask=mask)
    image_light = augmented['image']
    mask_light = augmented['mask']
    name=str(num)+str(k)+str(w)+str(j)+str(i)
    cv2.imwrite('AugFr/'+str(name)+'.png', image_light)
    cv2.imwrite('AugMasks/'+str(name)+'.png', mask_light)
    i=i+1
    if i==10:
        i=0
        j=j+1
    if j==10:
        j=0
        w=w+1
    if w==10:
        w=0
        k=k+1
    if k==10:
        k=0
        num=num+1
        
        
        
    # #10
    # aug = Compose([
    #     OneOf([RandomSizedCrop(min_max_height=(50, 101), height=original_height, width=original_width, p=0.5),
    #           PadIfNeeded(min_height=original_height, min_width=original_width, p=0.5)], p=1),    
    #     VerticalFlip(p=0.5),              
    #     RandomRotate90(p=0.5),
    #     OneOf([
    #         ElasticTransform(p=0.5, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
    #         GridDistortion(p=0.5),
    #         OpticalDistortion(p=1, distort_limit=1, shift_limit=0.5)                  
    #         ], p=0.8)])
    # augmented = aug(image=image, mask=mask)
    # image_medium = augmented['image']
    # mask_medium = augmented['mask']
    # name=str(num)+str(k)+str(w)+str(j)+str(i)
    # cv2.imwrite('AugFr/'+str(name)+'.png', image_medium)
    # cv2.imwrite('AugMasks/'+str(name)+'.png', mask_medium)
    # i=i+1
    # if i==10:
    #     i=0
    #     j=j+1
    # if j==10:
    #     j=0
    #     w=w+1
    # if w==10:
    #     w=0
    #     k=k+1
    # if k==10:
    #     k=0
    #     num=num+1

    # #11
    # aug = Compose([
    #     OneOf([RandomSizedCrop(min_max_height=(50, 101), height=original_height, width=original_width, p=0.5),
    #           PadIfNeeded(min_height=original_height, min_width=original_width, p=0.5)], p=1),    
    #     VerticalFlip(p=0.5),              
    #     RandomRotate90(p=0.5),
    #     OneOf([
    #         ElasticTransform(p=0.5, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
    #         GridDistortion(p=0.5),
    #         OpticalDistortion(p=1, distort_limit=2, shift_limit=0.5)                  
    #         ], p=0.8),
    #     CLAHE(p=0.8),
    #     RandomBrightnessContrast(p=0.8),    
    #     RandomGamma(p=0.8)])
    # augmented = aug(image=image, mask=mask)
    # image_heavy = augmented['image']
    # mask_heavy = augmented['mask']
    # name=str(num)+str(k)+str(w)+str(j)+str(i)
    # cv2.imwrite('AugFr/'+str(name)+'.png', image_heavy)
    # cv2.imwrite('AugMasks/'+str(name)+'.png', mask_heavy)
    # i=i+1
    # if i==10:
    #     i=0
    #     j=j+1
    # if j==10:
    #     j=0
    #     w=w+1
    # if w==10:
    #     w=0
    #     k=k+1
    # if k==10:
    #     k=0
    #     num=num+1


    #new import
    import albumentations as A

    BOX_COLOR = (255, 0, 0)
    TEXT_COLOR = (255, 255, 255)



    random.seed(42)
    #image = cv2.imread('images/parrot.jpg')

    light = A.Compose([
        A.RandomBrightnessContrast(p=1),    
        A.RandomGamma(p=1),    
        A.CLAHE(p=1),    
    ], p=1)

    medium = A.Compose([
        A.CLAHE(p=1),
        A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=50, val_shift_limit=50, p=1),
    ], p=1)


    strong = A.Compose([
        A.ChannelShuffle(p=1),
    ], p=1)


    # 12
    name=str(num)+str(k)+str(w)+str(j)+str(i)
    rr, mm, bb = augment_and_show(light, image, mask)
    cv2.imwrite('AugFr/'+str(name)+'.png', rr)
    cv2.imwrite('AugMasks/'+str(name)+'.png', mm)
    i=i+1
    if i==10:
        i=0
        j=j+1
    if j==10:
        j=0
        w=w+1
    if w==10:
        w=0
        k=k+1
    if k==10:
        k=0
        num=num+1


    #13
    name=str(num)+str(k)+str(w)+str(j)+str(i)
    rr, mm, bb = augment_and_show(medium, image, mask)
    cv2.imwrite('AugFr/'+str(name)+'.png', rr)
    cv2.imwrite('AugMasks/'+str(name)+'.png', mm)
    i=i+1
    if i==10:
        i=0
        j=j+1
    if j==10:
        j=0
        w=w+1
    if w==10:
        w=0
        k=k+1
    if k==10:
        k=0
        num=num+1


    #14
    name=str(num)+str(k)+str(w)+str(j)+str(i)
    rr, mm, bb = augment_and_show(strong, image, mask)
    cv2.imwrite('AugFr/'+str(name)+'.png', rr)
    cv2.imwrite('AugMasks/'+str(name)+'.png', mm)
    i=i+1
    if i==10:
        i=0
        j=j+1
    if j==10:
        j=0
        w=w+1
    if w==10:
        w=0
        k=k+1
    if k==10:
        k=0
        num=num+1



    light = A.Compose([   
        A.HorizontalFlip(p=1),
        A.RandomSizedCrop((450 - 50, 450 + 50), 480, 640),
        A.GaussNoise(var_limit=(100, 150), p=1),
    ], p=1)

    medium = A.Compose([
        A.HorizontalFlip(p=1),
        A.RandomSizedCrop((450 - 100, 450 + 50), 480, 640),
        A.MotionBlur(blur_limit=17, p=1),
    ], p=1)


    strong = A.Compose([
        A.HorizontalFlip(p=1),
        #A.RandomSizedCrop((800 - 100, 800 + 100), 600, 600),
        A.RGBShift(p=1),
        A.Blur(blur_limit=11, p=1),
        A.RandomBrightness(p=1),
        A.CLAHE(p=1),
    ], p=1)



    # #15
    # random.seed(42)
    # name=str(num)+str(k)+str(w)+str(j)+str(i)
    # rr, mm, bb = augment_and_show(light, image, mask)
    # cv2.imwrite('AugFr/'+str(name)+'.png', rr)
    # cv2.imwrite('AugMasks/'+str(name)+'.png', mm)
    # i=i+1
    # if i==10:
    #     i=0
    #     j=j+1
    # if j==10:
    #     j=0
    #     w=w+1
    # if w==10:
    #     w=0
    #     k=k+1
    # if k==10:
    #     k=0
    #     num=num+1



    # #16
    # name=str(num)+str(k)+str(w)+str(j)+str(i)
    # rr, mm, bb = augment_and_show(medium, image, mask)
    # cv2.imwrite('AugFr/'+str(name)+'.png', rr)
    # cv2.imwrite('AugMasks/'+str(name)+'.png', mm)
    # i=i+1
    # if i==10:
    #     i=0
    #     j=j+1
    # if j==10:
    #     j=0
    #     w=w+1
    # if w==10:
    #     w=0
    #     k=k+1
    # if k==10:
    #     k=0
    #     num=num+1



    #17
    name=str(num)+str(k)+str(w)+str(j)+str(i)
    rr, mm, bb = augment_and_show(strong, image, mask)
    cv2.imwrite('AugFr/'+str(name)+'.png', rr)
    cv2.imwrite('AugMasks/'+str(name)+'.png', mm)
    i=i+1
    if i==10:
        i=0
        j=j+1
    if j==10:
        j=0
        w=w+1
    if w==10:
        w=0
        k=k+1
    if k==10:
        k=0
        num=num+1


    #18
    # aug = Compose([
    #     OneOf([RandomSizedCrop(min_max_height=(50, 101), height=original_height, width=original_width, p=0.5),
    #           PadIfNeeded(min_height=original_height, min_width=original_width, p=0.5)], p=1),    
    #     VerticalFlip(p=0.5),              
    #     RandomRotate90(p=0.5),
    #     OneOf([
    #         ElasticTransform(p=0.5, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
    #         GridDistortion(p=0.5),
    #         OpticalDistortion(p=1, distort_limit=2, shift_limit=0.5)                  
    #         ], p=0.8),
    #     CLAHE(p=0.8),
    #     RandomBrightnessContrast(p=0.8),    
    #     RandomGamma(p=0.8)])
    # augmented = aug(image=image, mask=mask)
    # image_heavy = augmented['image']
    # mask_heavy = augmented['mask']
    # name=str(num)+str(k)+str(w)+str(j)+str(i)
    # cv2.imwrite('AugFr/'+str(name)+'.png', image_heavy)
    # cv2.imwrite('AugMasks/'+str(name)+'.png', mask_heavy)
    # i=i+1
    # if i==10:
    #     i=0
    #     j=j+1
    # if j==10:
    #     j=0
    #     w=w+1
    # if w==10:
    #     w=0
    #     k=k+1
    # if k==10:
    #     k=0
    #     num=num+1



    #19
    aug = Compose([VerticalFlip(p=0.5),              
                  RandomRotate90(p=0.5)])
    augmented = aug(image=image, mask=mask)
    image_light = augmented['image']
    mask_light = augmented['mask']
    name=str(num)+str(k)+str(w)+str(j)+str(i)
    cv2.imwrite('AugFr/'+str(name)+'.png', image_light)
    cv2.imwrite('AugMasks/'+str(name)+'.png', mask_light)
    i=i+1
    if i==10:
        i=0
        j=j+1
    if j==10:
        j=0
        w=w+1
    if w==10:
        w=0
        k=k+1
    if k==10:
        k=0
        num=num+1

    from urllib.request import urlopen



    #20
    # aug = A.RandomRain(p=1, brightness_coefficient=0.9, drop_width=1, blur_value=5)
    # name=str(num)+str(k)+str(w)+str(j)+str(i)
    # rr, mm, bb = augment_and_show(aug, image, mask)
    # cv2.imwrite('AugFr/'+str(name)+'.png', rr)
    # cv2.imwrite('AugMasks/'+str(name)+'.png', mm)
    # i=i+1
    # if i==10:
    #     i=0
    #     j=j+1
    # if j==10:
    #     j=0
    #     w=w+1
    # if w==10:
    #     w=0
    #     k=k+1
    # if k==10:
    #     k=0
    #     num=num+1


    #21
    # aug = A.RandomSnow(p=1, brightness_coeff=2.5, snow_point_lower=0.3, snow_point_upper=0.5)
    # name=str(num)+str(k)+str(w)+str(j)+str(i)
    # rr, mm, bb = augment_and_show(aug, image, mask)
    # cv2.imwrite('AugFr/'+str(name)+'.png', rr)
    # cv2.imwrite('AugMasks/'+str(name)+'.png', mm)
    # i=i+1
    # if i==10:
    #     i=0
    #     j=j+1
    # if j==10:
    #     j=0
    #     w=w+1
    # if w==10:
    #     w=0
    #     k=k+1
    # if k==10:
    #     k=0
    #     num=num+1



    # #22
    # aug = A.RandomSunFlare(p=1.0, flare_roi=(0, 0, 1, 0.5), angle_lower=0.5)
    # name=str(num)+str(k)+str(w)+str(j)+str(i)
    # rr, mm, bb = augment_and_show(aug, image, mask)
    # cv2.imwrite('AugFr/'+str(name)+'.png', rr)
    # cv2.imwrite('AugMasks/'+str(name)+'.png', mm)
    # i=i+1
    # if i==10:
    #     i=0
    #     j=j+1
    # if j==10:
    #     j=0
    #     w=w+1
    # if w==10:
    #     w=0
    #     k=k+1
    # if k==10:
    #     k=0
    #     num=num+1



    # #23
    # aug = A.RandomFog(p=1, fog_coef_lower=0.7, fog_coef_upper=0.8, alpha_coef=0.1)
    # name=str(num)+str(k)+str(w)+str(j)+str(i)
    # rr, mm, bb = augment_and_show(aug, image, mask)
    # cv2.imwrite('AugFr/'+str(name)+'.png', rr)
    # cv2.imwrite('AugMasks/'+str(name)+'.png', mm)
    # i=i+1
    # if i==10:
    #     i=0
    #     j=j+1
    # if j==10:
    #     j=0
    #     w=w+1
    # if w==10:
    #     w=0
    #     k=k+1
    # if k==10:
    #     k=0
    #     num=num+1




    from albumentations import (
        HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
        Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
        IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
        IAASharpen, IAAEmboss, Flip, OneOf, Compose
    )

    #24
    aug = ShiftScaleRotate(p=1)
    name=str(num)+str(k)+str(w)+str(j)+str(i)
    rr, mm, bb = augment_and_show(aug, image, mask)
    cv2.imwrite('AugFr/'+str(name)+'.png', rr)
    cv2.imwrite('AugMasks/'+str(name)+'.png', mm)
    i=i+1
    if i==10:
        i=0
        j=j+1
    if j==10:
        j=0
        w=w+1
    if w==10:
        w=0
        k=k+1
    if k==10:
        k=0
        num=num+1




    #25
    aug = augment_flips_color(p=1)
    name=str(num)+str(k)+str(w)+str(j)+str(i)
    rr, mm, bb = augment_and_show(aug, image, mask)
    cv2.imwrite('AugFr/'+str(name)+'.png', rr)
    cv2.imwrite('AugMasks/'+str(name)+'.png', mm)
    i=i+1
    if i==10:
        i=0
        j=j+1
    if j==10:
        j=0
        w=w+1
    if w==10:
        w=0
        k=k+1
    if k==10:
        k=0
        num=num+1



    #26
    aug = strong_aug(p=1)
    name=str(num)+str(k)+str(w)+str(j)+str(i)
    rr, mm, bb = augment_and_show(aug, image, mask)
    cv2.imwrite('AugFr/'+str(name)+'.png', rr)
    cv2.imwrite('AugMasks/'+str(name)+'.png', mm)
    i=i+1
    if i==10:
        i=0
        j=j+1
    if j==10:
        j=0
        w=w+1
    if w==10:
        w=0
        k=k+1
    if k==10:
        k=0
        num=num+1
    #if index>2:
        #break
    print(index)



In [ ]:
num_of_pics = 2000*18
print(num_of_pics)
end_index = num_of_pics + 1999
print(end_index)
total = end_index + 1
print(total)